In [1]:
import sys

import pygame as pg
from pygame.locals import *
import numpy as np

from car import Car
from track import Track

import neat
import math
import time
import pickle

pygame 2.5.2 (SDL 2.28.3, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
maps = [
    {'name': 'track01', 'file': 'tracks/track01.png', 'x_text': 1920 / 2, 'y_text': 1080 / 2, 'distance_text': 50},
    {'name': 'track02', 'file': 'tracks/track02.png', 'x_text': 1716, 'y_text': 400, 'distance_text': 50},
    {'name': 'track03', 'file': 'tracks/track03.png', 'x_text': 300, 'y_text': 810, 'distance_text': 50},
]

In [3]:
class NeatCar(Car):
    MAX_LENGTH = 200

    def __init__(self, car_sprite, pos_x, pos_y, angle, speed, game_map, border_color, map_width, map_height,
                 top_start_line, bottom_start_line):
        super().__init__(car_sprite, pos_x, pos_y, angle, speed, game_map, border_color, map_width, map_height,
                         top_start_line, bottom_start_line)
        self.radars = []
        self.speed_changes = 0
        self.speed_incremental_average = 1
        self.angles = []

    def get_reward(self):
        return self.distance / (Car.CAR_SIZE_X / 2) + self.has_touched_finish() * 10000

    def check_radar(self, degree):
        def compute_x(_length):
            return int(self.center[0] + _length * math.cos(math.radians(360 - (self.angle + degree))))

        def compute_y(_length):
            return int(self.center[1] + _length * math.sin(math.radians(360 - (self.angle + degree))))

        length = 0
        x = compute_x(length)
        y = compute_y(length)

        while not self.is_collision_points(x, y) and length < NeatCar.MAX_LENGTH:
            length += 1
            x = compute_x(length)
            y = compute_y(length)

        dist = int(pg.math.Vector2(x, y).distance_to(self.center))
        self.radars.append([(x, y), dist])

        if degree == 0:
            length = 0
            x = compute_x(length)
            y = compute_y(length)

            while not self.is_collision_points(x, y) and length < NeatCar.MAX_LENGTH + 100:
                length += 1
                x = compute_x(length)
                y = compute_y(length)

            dist = int(pg.math.Vector2(x, y).distance_to(self.center))
            self.radars.append([(x, y), dist])

    def draw(self, screen):
        super().draw(screen)
        for r in self.radars:
            pg.draw.line(screen, (0, 255, 0), self.center, r[0], 1)
            pg.draw.circle(screen, (0, 255, 0), r[0], 5)

    def update(self):
        super().update()

        self.speed_incremental_average = (self.speed_incremental_average * self.speed_changes + self.speed) / (
                self.speed_changes + 1)

        self.radars.clear()
        for d in range(-90, 120, 30):
            self.check_radar(d)

    def get_neat_data(self):
        return [r[1] // 30 for r in self.radars] if len(self.radars) > 0 else [0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
def run_simulation(genomes, config, speed=20, full_screen=False):
    # Initialize networks and cars lists to keep track of each genome's neural network and the associated car
    networks = []
    cars = []

    # Initialize Pygame
    pg.init()

    # Load the current map
    global current_map_path
    my_track = Track(current_map['file'])

    # Set up the display mode based on whether full screen is enabled or not
    screen = pg.display.set_mode((my_track.map_width, my_track.map_height),
                                 (SCALED | RESIZABLE) if not full_screen else FULLSCREEN)
    my_track.load_game_map()  # Load the game map
    top_start_line, bottom_start_line = my_track.get_start_line_points()  # Get the start line positions

    # Initialize a clock for controlling the frame rate
    clock = pg.time.Clock()

    # Calculate the starting position and orientation for the cars
    start_pos_x, start_pos_y, angle = my_track.start_pos
    start_pos_y -= Car.CAR_SIZE_Y // 2  # Adjust starting Y position based on car size

    # Initialize each genome and its corresponding neural network and car
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)  # Create a neural network for the genome
        networks.append(net)
        g.fitness = 0  # Initialize fitness to 0
        # Create a car with the specified parameters and add it to the cars list
        cars.append(
            NeatCar('cars/car2d.png', start_pos_x, start_pos_y, angle, speed, my_track.game_map, my_track.border_color,
                    my_track.width, my_track.height, top_start_line, bottom_start_line))

    # Increment the generation counter
    global current_generation
    current_generation += 1

    # Set up a timer for the simulation run
    timer = time.time()
    MAX_TIME = 60  # Maximum time for the simulation in seconds

    # Main simulation loop
    while True:
        # Process Pygame events
        for event in pg.event.get():
            if event.type == QUIT:
                pg.quit()

        no_still_alive = 0  # Counter for the number of cars still alive

        # Update each car based on its neural network's output
        for i, car in enumerate(cars):
            output = networks[i].activate(car.get_neat_data())  # Get the neural network's output
            choice = output.index(max(output))  # Choose the action with the highest output value

            car.move(choice)  # Move the car based on the chosen action

            if car.is_alive():  # Check if the car is still alive
                no_still_alive += 1
                car.update()  # Update the car's state
                genomes[i][1].fitness = car.get_reward()  # Update the genome's fitness based on the car's reward

        # End the simulation if no cars are alive or the maximum time has been exceeded
        if no_still_alive == 0 or time.time() - timer > MAX_TIME:
            break

        # Draw the game map and all alive cars
        screen.blit(my_track.game_map, (0, 0))
        for car in cars:
            if car.is_alive():
                car.draw(screen)

        # Display the current generation and the number of cars still alive
        display_text(screen, no_still_alive)

        # Update the display and maintain the frame rate
        pg.display.flip()
        clock.tick(60)  # Target 60 frames per second

    # Uncomment to quit Pygame when the simulation loop ends
    # pg.quit()

def display_text(screen, no_still_alive):
    # Set up fonts for displaying text
    font_generation = pg.font.SysFont("Arial", 30)
    font_alive = pg.font.SysFont("Arial", 20)

    # Render the text for the current generation and number of cars still alive
    text_gen = font_generation.render("Generation: " + str(current_generation), True, (0, 0, 0))
    text_gen_rect = text_gen.get_rect()
    text_gen_rect.center = (current_map['x_text'], current_map['y_text'])
    screen.blit(text_gen, text_gen_rect)

    text_alive = font_alive.render("Alive: " + str(no_still_alive), True, (0, 0, 0))
    text_alive_rect = text_alive.get_rect()
    text_alive_rect.center = (current_map['x_text'], current_map['y_text'] + current_map['distance_text'])
    screen.blit(text_alive, text_alive_rect)


In [5]:
max_fitness = []

for map in maps:
    current_map = map

    current_generation = 0

    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet,
                                neat.DefaultStagnation, 'alg_neat/neat.config')
    population = neat.Population(config)
    population.add_reporter(neat.StdOutReporter(True))

    stats = neat.StatisticsReporter()
    population.add_reporter(stats)

    population.run(run_simulation, 30)
    pg.quit()

    with open(f'alg_neat/best_genome_{map["name"]}.pkl', 'wb') as output:
        pickle.dump(population, output, 1)

    best_genome = stats.best_genome()
    max_fitness.append(best_genome.fitness)

print(max_fitness)

Exception: No such config file: D:\PythonProjects\F1_RL\models\alg_neat\neat.config

In [ ]:
for i in range(len(maps)):
    current_map = maps[i]
    for j in range(len(maps)):
        if i == j:
            continue

        with open(f'alg_neat/best_genome_{maps[j]["name"]}.pkl', 'rb') as input:
            best_genome = pickle.load(input).best_genome

        run_simulation([(1, best_genome)], config, speed=20, full_screen=False)

pg.quit()